In [2]:
import pandas as pd
from datetime import datetime, timedelta

### Load the CSV data 

In [10]:
df = pd.read_csv('ipdr.csv')
df.head()

,starttime,endtime,msisdn,ulvolume,dlvolume,domain
0,2021-04-0212:23:10,2021-04-0212:24:48,1,10819,9960,app1
1,2021-04-0212:28:56,2021-04-0212:33:12,1,16067,10663,app1
2,2021-04-0216:24:21,2021-04-0216:38:28,2,1173,4265,app1
3,2021-04-0215:08:57,2021-04-0215:20:56,2,1200,192,app2
4,2021-04-0215:08:57,2021-04-0215:20:41,2,175130,101657,app2


### Function to convert string to datetime

In [5]:
def to_datetime(dt_str):
    return datetime.strptime(dt_str, '%Y-%m-%d%H:%M:%S')

### Convert starttime and endtime to datetime objects

In [11]:
df['starttime'] = df['starttime'].apply(to_datetime)
df['endtime'] = df['endtime'].apply(to_datetime)
df.head()

,starttime,endtime,msisdn,ulvolume,dlvolume,domain
0,2021-04-02 12:23:10,2021-04-02 12:24:48,1,10819,9960,app1
1,2021-04-02 12:28:56,2021-04-02 12:33:12,1,16067,10663,app1
2,2021-04-02 16:24:21,2021-04-02 16:38:28,2,1173,4265,app1
3,2021-04-02 15:08:57,2021-04-02 15:20:56,2,1200,192,app2
4,2021-04-02 15:08:57,2021-04-02 15:20:41,2,175130,101657,app2


### Function to calculate the adjusted ET (ET*(ET-10min)) for each FDR

In [8]:
def adjust_endtime(row):
    st = row['starttime']
    et = row['endtime']
    
    # Calculate the adjusted ET
    adjusted_et = et - timedelta(minutes=10)
    
    # Apply the condition: if adjusted ET is less than ST, keep original ET
    if adjusted_et < st:
        return et  # Keep the original ET
    else:
        return adjusted_et  # Use adjusted ET

### Apply the function to adjust the end times

In [20]:
df['adjusted_endtime'] = df.apply(adjust_endtime, axis=1)
df.head()

,starttime,endtime,msisdn,ulvolume,dlvolume,domain,total_volume_kb,adjusted_endtime
0,2021-04-02 12:23:10,2021-04-02 12:24:48,1,10819,9960,app1,20.291992,2021-04-02 12:24:48
1,2021-04-02 12:28:56,2021-04-02 12:33:12,1,16067,10663,app1,26.103516,2021-04-02 12:33:12
2,2021-04-02 16:24:21,2021-04-02 16:38:28,2,1173,4265,app1,5.310547,2021-04-02 16:28:28
3,2021-04-02 15:08:57,2021-04-02 15:20:56,2,1200,192,app2,1.359375,2021-04-02 15:10:56
4,2021-04-02 15:08:57,2021-04-02 15:20:41,2,175130,101657,app2,270.299805,2021-04-02 15:10:41


### Calculate total volume (DL + UL) in kilobytes

In [21]:
df['total_volume_kb'] = (df['ulvolume'] + df['dlvolume']) / 1024  # Convert bytes to KB
df.head()

,starttime,endtime,msisdn,ulvolume,dlvolume,domain,total_volume_kb,adjusted_endtime
0,2021-04-02 12:23:10,2021-04-02 12:24:48,1,10819,9960,app1,20.291992,2021-04-02 12:24:48
1,2021-04-02 12:28:56,2021-04-02 12:33:12,1,16067,10663,app1,26.103516,2021-04-02 12:33:12
2,2021-04-02 16:24:21,2021-04-02 16:38:28,2,1173,4265,app1,5.310547,2021-04-02 16:28:28
3,2021-04-02 15:08:57,2021-04-02 15:20:56,2,1200,192,app2,1.359375,2021-04-02 15:10:56
4,2021-04-02 15:08:57,2021-04-02 15:20:41,2,175130,101657,app2,270.299805,2021-04-02 15:10:41


### Group by msisdn and domain to identify each call

In [22]:
calls = df.groupby(['msisdn', 'domain'])
calls.head()

,starttime,endtime,msisdn,ulvolume,dlvolume,domain,total_volume_kb,adjusted_endtime
0,2021-04-02 12:23:10,2021-04-02 12:24:48,1,10819,9960,app1,20.291992,2021-04-02 12:24:48
1,2021-04-02 12:28:56,2021-04-02 12:33:12,1,16067,10663,app1,26.103516,2021-04-02 12:33:12
2,2021-04-02 16:24:21,2021-04-02 16:38:28,2,1173,4265,app1,5.310547,2021-04-02 16:28:28
3,2021-04-02 15:08:57,2021-04-02 15:20:56,2,1200,192,app2,1.359375,2021-04-02 15:10:56
4,2021-04-02 15:08:57,2021-04-02 15:20:41,2,175130,101657,app2,270.299805,2021-04-02 15:10:41
5,2021-04-02 15:09:22,2021-04-02 15:21:02,2,1440,0,app2,1.406250,2021-04-02 15:11:02
6,2021-04-02 15:09:22,2021-04-02 15:20:35,2,3672,1152,app2,4.710938,2021-04-02 15:10:35
7,2021-04-02 15:09:22,2021-04-02 15:21:08,2,1440,0,app2,1.406250,2021-04-02 15:11:08
8,2021-04-02 15:54:46,2021-04-02 16:11:49,3,10932,5504,app3,16.050781,2021-04-02 16:01:49
9,2021-04-02 15:54:46,2021-04-02 15:55:34,3,500,580,app3,1.054688,2021-04-02 15:55:34


### Process each group (call)

In [23]:
# Store results
results = []

for (msisdn, domain), group in calls:
    # Sort by start time within each group
    group = group.sort_values(by='starttime')
    
    # Calculate the total volume (sum of all FDRs)
    total_volume_kb = group['total_volume_kb'].sum()
    
    # Calculate the total time in seconds (from the earliest ST to the latest adjusted ET)
    total_time_sec = (group['adjusted_endtime'].max() - group['starttime'].min()).total_seconds()
    
    # Calculate the bit rate (kbps)
    bit_rate_kbps = total_volume_kb / (total_time_sec / 1000) if total_time_sec > 0 else 0
    
    # Determine if the call is audio or video based on the bit rate
    is_audio = bit_rate_kbps <= 200
    is_video = bit_rate_kbps > 200
    
    # Count the number of FDRs (CDRs) in the call
    fdr_count = len(group)
    
    # Append the result for this call
    results.append({
        'msisdn': msisdn,
        'domain': domain,
        'duration_sec': total_time_sec,
        'fdr_count': fdr_count,
        'bit_rate_kbps': bit_rate_kbps,
        'isAudio': is_audio,
        'isVideo': is_video
    })

### Convert results to a DataFrame

In [24]:
results_df = pd.DataFrame(results)
results_df.head()

,msisdn,domain,duration_sec,fdr_count,bit_rate_kbps,isAudio,isVideo
0,1,app1,602.0,2,77.068950,True,False
1,2,app1,247.0,1,21.500190,True,False
2,2,app2,131.0,5,2131.165017,False,True
3,3,app3,423.0,6,9018.589317,False,True
4,3,app4,20032.0,4,795.740292,False,True


### Save results to CSV.

In [26]:
results_df.to_csv('voip_call_output.csv', index=False)